In [1]:
import pandas as pd
import openai
from dotenv import load_dotenv
load_dotenv()
import os
from langchain.prompts import load_prompt
from langchain import PromptTemplate
from langchain.llms import OpenAIChat
import sys
pd.options.mode.chained_assignment = None
from langchain.callbacks import get_openai_callback
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain import PromptTemplate, LLMChain, VectorDBQA
from langchain.chains import TransformChain, SequentialChain
from langchain.document_loaders.csv_loader import CSVLoader
import re
import json
import stripe

from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    SystemMessage
)
import asyncio


import sys
sys.path.append('./modules/')
openai.api_key = os.getenv('OPENAI_KEY')
key = os.getenv('OPENAI_KEY')

from Generator import Generator
from Extractor import Extractor
from PromptPath import PromptPath
from GenExtract import GenExtract
from Property import Property
from NaturalLanguagePool import NaturalLanguagePool
from DynamicChatHistory import DynamicChatHistory
from Conversation import Conversation
from ConversationScript import ConversationScript
from ConversationScriptStage import ConversationScriptStage
from show_chat_log import show_chat_log
from is_question import is_question
from customer_profile import customer_profile

times_embedding_run = 10

RED = "\033[91m"
GREEN = "\033[92m"
RESET = "\033[0m"
BOLD = "\033[1m"

In [2]:
chat = ChatOpenAI(openai_api_key=key)

### Create the embeddings

In [3]:
#load in the policies to the vectorstore
situation_loaders = []
situation_docs = []

#create a list with the names of the files to be loaded in the data/policies folder
for file in os.listdir('data/situational-experience'):
    if file.endswith('.txt'):
        loader = TextLoader('data/situational-experience/' + file)
        situation_loaders.append(loader)

for loader in situation_loaders:
    situation_docs.extend(loader.load())

if times_embedding_run < 20:
    text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
    documents = text_splitter.split_documents(situation_docs)

    siutation_embeddings = OpenAIEmbeddings(openai_api_key=key)
    situational_experience = Chroma.from_documents(documents, siutation_embeddings)
    times_embedding_run += 1


"Embeddings generated into policies variable"

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


'Embeddings generated into policies variable'

In [4]:
#iterate over the contents of data/services and load them into the vectorstore
service_loaders = []
service_docs = []

#create a list with the names of the files to be loaded in the data/services folder
for file in os.listdir('data/services'):
    if file.endswith('.json'):
        loader = TextLoader('data/services/' + file)
        service_loaders.append(loader)

for loader in service_loaders:
    service_docs.extend(loader.load())

service_embeddings = OpenAIEmbeddings(openai_api_key=key)
service_description_search = Chroma.from_documents(service_docs, service_embeddings)
    

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


In [5]:
all_services_df = pd.read_csv('data/services_search - Sheet1.csv')

class Programs:
    def __init__(self, services):
        self.services = services

    def get_service(self, name):
        for service in self.services:
            if service['Name'] == name:
                return service
        return None
    
class Program:
    def __init__(self, dict):
        self.dict = dict

#create a list of dicts out of the contents of all_services_df
all_services = []
for index, row in all_services_df.iterrows():
    all_services.append(row.to_dict())

all_services_list = Programs(all_services)

#### **CHAIN**: Retrieve relevant experiences from the vectorstore

In [6]:
def get_situational_awareness(inputs: dict) -> dict:
    print(GREEN + 'STEP: Getting situational experiences' + RESET)
    customer_issue = inputs["customer_issue"]
    knowledge = situational_experience.similarity_search(customer_issue, 5)
    knowledge = [message.page_content for message in knowledge]
    print(RED + 'Situational experiences found: ', knowledge, RESET) 
    print(GREEN + 'STEP: Analyzing situational experiences' + RESET)       
    return { "situational_experience": knowledge }

retrieve_situational_awareness = TransformChain(transform=get_situational_awareness, input_variables=["customer_issue"], output_variables=["situational_experience"])

#### **CHAIN**: Analyze the experiences we've had

In [7]:
template_message = """Your goal is to identify how severe this issue is. Think step by step about your answer and use the information you provided to make you answer. After your analysis, please rate the severity as Low, Medium or High.
Customer Issue: {customer_issue}
Property Type: {property_type}
Below are some possible practical experiences from our technicians that may be useful. Some of it may not be relevant to your answer.
{situational_experience}
----
Your Analysis:
"""

template = PromptTemplate(template=template_message, input_variables=["customer_issue", "situational_experience", "property_type"])
analyze_issue_chain = LLMChain(llm=chat, prompt=template, output_key="siutation_analysis")

In [8]:
def get_relevant_services(inputs: dict) -> dict:
    analysis = inputs["siutation_analysis"]
    print(GREEN + 'STEP: Getting relevant services' + RESET)
    service_search_results = service_description_search.similarity_search(analysis, 3)
    #extract the services as a dict from the search results (they are json stored in .page_content)
    service_search_results = [json.loads(result.page_content) for result in service_search_results]
    detailed_services = []
    for service in service_search_results:
        service_name = service['Name']
        detailed_service = all_services_list.get_service(service_name)
        detailed_services.append(Program(detailed_service))

    relevant_services_text = ""
    for service in detailed_services:
        relevant_services_text += service.dict['Name'] + ": "
        relevant_services_text += service.dict['Description'] + "\n"
        relevant_services_text += "Best for: " + service.dict['Best for'] + "\n"
        relevant_services_text += "Covers: " + service.dict['Covered Pests'] + "\n"
        relevant_services_text += "Protection: " + service.dict['Level of Protection'] + "\n"
        relevant_services_text += "Price: " + service.dict['Cost'] + "\n\n"
    
    print(RED + 'STEP: Completed getting services: ', detailed_services, RESET)
    print(GREEN + 'STEP: Begin parsing the formulas' + RESET)

    for service in detailed_services:
        recurring_formula = service.dict['Recurring Formula']
        recurring_formula = recurring_formula.replace("{square_footage}", str(inputs["square_footage"]))
        recurring_formula = recurring_formula.replace("{target}", str(inputs["target"]))
        recurring_formula = recurring_formula.replace("{severity}", str(inputs["severity"]))
        recurring_formula = recurring_formula.replace("{acres}", str(inputs["acres"]))
        service.dict['Recurring Formula'] = recurring_formula

        initial_formula = service.dict['Initial Formula']
        initial_formula = initial_formula.replace("{square_footage}", str(inputs["square_footage"]))
        initial_formula = initial_formula.replace("{target}", str(inputs["target"]))
        initial_formula = initial_formula.replace("{severity}", str(inputs["severity"]))
        initial_formula = initial_formula.replace("{acres}", str(inputs["acres"]))
        service.dict['Initial Formula'] = initial_formula
    
    print(GREEN + 'STEP: Completed parsing the formulas' + RESET)

    return { "relevant_services_dict": detailed_services, "relevant_services_text": relevant_services_text }

get_relevant_services_chain = TransformChain(transform=get_relevant_services, input_variables=["siutation_analysis"], output_variables=["relevant_services_dict", "relevant_services_text"])

In [36]:
from langchain.chains import APIChain
import nest_asyncio


calculator_template = """Solve this excel formula. Briefly show your thinking for each step. Then write the answer as:
Answer: *answer*
Example:
=if(1=2, 1, 2) + 1
Thinking: 1=2 is false so the answer is 2 + 1
Answer: 3
Formula:
{formula}
"""

#majority vote will take in a list of results and return the most common answer for each type and name (ex. item: {'name': 'Service 1', 'answer': '5', 'type': 'recurring', 'iter_num': 0})
def majority_vote(results):
    # Group by name and type
    grouped_results = {}
    for result in results:
        name = result['name']
        type = result['type']
        if name not in grouped_results:
            grouped_results[name] = {}
        if type not in grouped_results[name]:
            grouped_results[name][type] = []
        grouped_results[name][type].append(result)

    # Find the most common answer for each name and type, excluding None unless it's the only result
    majority_vote_results = []
    for name in grouped_results:
        for type in grouped_results[name]:
            answers = [result['answer'] for result in grouped_results[name][type]]
            
            # Exclude None values if there are other non-None values
            non_none_answers = [answer for answer in answers if answer is not None]
            if non_none_answers:
                answers = non_none_answers

            answer = max(set(answers), key=answers.count)
            majority_vote_results.append({'name': name, 'answer': answer, 'type': type})

    return majority_vote_results


async def async_calculate_price(name, chain, formula, type, iter=1, timeout=8):
    max_retries = 3
    retries = 0
    answer = None
    
    while retries < max_retries:
        try:
            resp = await asyncio.wait_for(chain.arun(formula=formula), timeout=timeout)
            print(BOLD, resp, RESET)
            # parse the Answer: *answer* from the response using regex
            answer = re.search(r'Answer:\s*\$?(\d+(?:\.\d+)?)\s*[^\d\(\)]*', resp)
            
            #if there is a match, break the loop
            if answer is not None:
                answer = answer.group(1)
                break
        except asyncio.TimeoutError:
            print(RED + f"STEP: Timeout reached for {formula} after {timeout} seconds (Retry {retries + 1})" + RESET)
            retries += 1
    
    if answer is None:
        print(RED + f"STEP: Failed to parse answer from response after {max_retries} retries" + RESET)
    
    return { "name": name, "answer": answer, "type": type, "iter_num": iter }

async def get_pricing(inputs: dict) -> dict:
    print(GREEN + 'STEP: Generating recurring price' + RESET)
    services = inputs["relevant_services_dict"]
    tasks = []
    for service in services:
        name = service.dict['Name']
        recurring_formula = service.dict['Recurring Formula']
        intial_formula = service.dict['Initial Formula']
        calculator_prompt = PromptTemplate(
            template=calculator_template,
            input_variables=["formula"]
        )
        chain = LLMChain(llm=chat, prompt=calculator_prompt, output_key="recurring_logic")
        #majority vote
        for n in range(1, 3):
            recurring_task = asyncio.create_task(async_calculate_price(name, chain, recurring_formula, 'recurring', iter=n))
            initial_task = asyncio.create_task(async_calculate_price(name, chain, intial_formula, 'initial', iter=n))
            tasks.append(recurring_task)
            tasks.append(initial_task)

    print(GREEN + 'All Tasks Added to Queue' + RESET)
    results = await asyncio.gather(*tasks)
    print(GREEN, 'All Tasks Completed' + RESET)
    print(GREEN, 'Holding majority vote' + RESET)
    results = majority_vote(results)
    print(GREEN, 'Majority vote complete' + RESET)

    for service in services:
        name = service.dict['Name']
        for result in results:
            if name == result['name'] and result['type'] == 'recurring':
                service.dict['Recurring Price'] = result['answer']
            if name == result['name'] and result['type'] == 'initial':
                service.dict['Initial Price'] = result['answer']
    
    service_offers_formatted = ""
    for service in services:
        service_offers_formatted += service.dict['Outward Facing'] + '\n'
        service_offers_formatted += 'Initial Price: $' + service.dict['Initial Price'] + '\n'
        service_offers_formatted += 'Recurring Price: $' + service.dict['Recurring Price'] + '\n'
        service_offers_formatted += 'Description: ' + service.dict['Description'] + '\n'
        service_offers_formatted += 'Covered Pests: ' + service.dict['Covered Pests'] + '\n'
        service_offers_formatted += 'Best For: ' + service.dict['Best for'] + '\n'
        service_offers_formatted += 'Frequency: ' + service.dict['Frequency'] + '\n'
        service_offers_formatted += 'Level of Protection: ' + service.dict['Level of Protection'] + '\n'
        service_offers_formatted += 'Benefits: ' + service.dict['Benefits'] + '\n'
        service_offers_formatted += 'Payment Link: <pay:' + service.dict['Name'] + '>\n'
        service_offers_formatted += '\n'

    return { "complete_services": services, "service_offers_formatted": service_offers_formatted }

def add_prices(inputs):
    loop = asyncio.get_event_loop()
    nest_asyncio.apply(loop)
    response = loop.run_until_complete(get_pricing(inputs))
    return { "complete_services": response["complete_services"], "service_offers_formatted": response['service_offers_formatted'] }


calculate_prices_chain = TransformChain(
    transform=add_prices, 
    input_variables=["relevant_services_dict"], 
    output_variables=["complete_services", "service_offers_formatted"]
)

In [50]:
pitch_template = """You are an expert and friendly salesperson for a sales company. You have just finished collecting information from the customer. Provide a helpful and entertaining explaination of the best service you found for them based on the following service and issue. Focus on service benefits and avoid being overly technical. Be casual and pretend you are chatting with a friend. At the end include the payment link. 
Customer Issue: {customer_issue}
Customer Preferences: {customer_preferences}
Your Analysis: {siutation_analysis}
Conversation So Far: 
{conversation_history}
Services you can choose from: 
{service_offers_formatted}
Your pitch:
"""

parser_prompt = PromptTemplate(template=pitch_template, input_variables=["conversation_history", "customer_issue", "customer_preferences", "service_offers_formatted", "siutation_analysis"])
sales_pitch_chain = LLMChain(llm=chat, prompt=parser_prompt, output_key="pitch")

In [60]:
class Payments:
    def __init__(self, api_key):
        self.api_key = api_key
    
    def create_payment_link(self, service_id, initial):
        stripe.api_key = self.api_key
        unit_amount = int(initial * 100)
        #create the price for the service in the stripe dashboard
        price = stripe.Price.create(currency="usd", unit_amount=unit_amount, product=service_id)
        price_id = price.id
        #create the payment link
        link = stripe.checkout.Session.create(
            payment_method_types=['card'],
            line_items=[
                {
                    'price': price_id,
                    'quantity': 1,
                },
            ],
            mode='payment',
            success_url='https://example.com/success',
            cancel_url='https://example.com/cancel',
        )
        return link.url


pay = Payments(api_key=os.getenv('STRIPE_KEY'))

def add_payment_links(inputs):
    services = inputs["complete_services"]
    updated_services = []
    for service in services:
        service_id = service.dict['Service ID']
        initial = float(service.dict['Initial Price'])
        if initial == 0:
            initial = float(service.dict['Recurring Price'])
        link = pay.create_payment_link(service_id, initial)
        service.dict['Payment Link'] = link
        updated_services.append(service)

    return { "complete_services_with_links": updated_services }

payment_links_chain = TransformChain(
    transform=add_payment_links,
    input_variables=["complete_services"],
    output_variables=["complete_services_with_links"]
)

In [61]:
def parse_pitch(inputs):
    pitch = inputs["pitch"]
    #find the string in the form of <pay:service_name>
    payment_link = re.search(r'<pay:(.*?)>', pitch)
    if payment_link:
        print(GREEN, 'Found payment link' + RESET)
        program_name = payment_link.group(1).strip()
    else:
        print(RED, 'No payment link found' + RESET)
        program_name = ''
    #find the service that matches the name
    services = inputs["complete_services_with_links"]
    payment_link = "http://error.com"
    for service in services:
        if service.dict['Name'] == program_name:
            payment_link = service.dict['Payment Link']
    
    #replace the string with the payment link
    pitch = re.sub(r'<pay:(.*?)>', payment_link, pitch)
    return { "final pitch": pitch }

parse_pitch_chain = TransformChain(
    transform=parse_pitch,
    input_variables=["pitch", "complete_services"],
    output_variables=["final pitch"]
)

In [62]:
conversation = """
"""

analyze_situation_sequential = SequentialChain(
    chains=[
        retrieve_situational_awareness,
        analyze_issue_chain,
        get_relevant_services_chain,
        calculate_prices_chain,
        payment_links_chain,
        sales_pitch_chain,
        parse_pitch_chain
    ],
    input_variables=["customer_issue", "customer_preferences", "square_footage", "property_type", "acres", "target", "severity", "conversation_history"],
    output_variables=[
        "situational_experience", 
        "siutation_analysis", 
        "relevant_services_dict",
        "complete_services",
        "service_offers_formatted",
        "pitch",
        "final pitch"
    ],
    verbose=True
)

In [63]:
conversation = """
Lead: I have a problem with ants in my kitchen.
Agent: Oh no! I'm sorry to hear that John. Luckily we can help with that :) Would you mind if I asked you a few questions so I can get a handle on your issue?
Lead: Sure, go ahead.
Agent: Great thanks! First how long has this been going on?
Lead: 2 Weeks
Agent: And how many ants are you seeing?
Lead: A lot! They are all over the kitchen.
Agent: OK don't worry I see this kind of thing all the time. How big is your house?
Lead: 2000 square feet
Agent: Got it. Give me just a second and let me run some numbers for you.
Agent: OK here's what I'm thinking:
"""

In [64]:
response = analyze_situation_sequential({
    "customer_issue": "ants in kitchen",
    "customer_preferences": "none stated",
    "property_type": "single family",
    "square_footage": 2500,
    "acres": 1,
    "target": "ants",
    "severity": "high",
    "conversation_history": conversation
})



> Entering new SequentialChain chain...
STEP: Getting situational experiences
Situational experiences found:  ['Carpenter ants are usually harder to control than ants and will require bimonthly or monthly services.', 'Ants can be well managed by bimonthly service frequencies. It is less advisable, but still an option to allow customers to use quarterly services. Monthly services are often overkill for ant issues.', 'Program 1 for termites does not work well in townhouses or apartments'] 
STEP: Analyzing situational experiences
STEP: Getting relevant services
STEP: Completed getting services:  [<__main__.Program object at 0x14cf06310>, <__main__.Program object at 0x16808c1f0>, <__main__.Program object at 0x1680124c0>] 
STEP: Begin parsing the formulas
STEP: Completed parsing the formulas
STEP: Generating recurring price
All Tasks Added to Queue
 Thinking: The first condition, "medium"="high", is false so it moves to the nested if statement. The second condition, "medium"="medium" is t

In [65]:
print(response['final pitch'])

Hey John, based on our conversation, I think the Castle Program would be the best service for you to handle those pesky ants in your kitchen. The service offers bimonthly pest control to cover the ant issue, as well as millipedes, crickets, and centipedes. With this program, you'll have proactive annual termite inspections to help reduce the risk of undetected termites in your home. Plus, you'll get year-round protection against 33 pests, with a level of protection that's average, but still more than enough for your current issue. And the best part? You'll have the convenience of a single bill on autopilot. 

I hope this helps, John! We take pride in offering affordable and effective pest control services to our customers, and I'm confident that the Castle Program will take care of all your ant troubles. Let me send you the payment link right here: https://checkout.stripe.com/c/pay/cs_test_a1PytlOBsB3AyUqNOOaNI7EakaWfzai5EW0iGT1zMUaqilmOssAilJ9jYY#fidkdWxOYHwnPyd1blpxYHZxWjA0TTJoRHZOaG

In [54]:
example = ['customer_issue',
 'customer_preferences',
 'property_type',
 'square_footage',
 'acres',
 'target',
 'severity',
 'situational_experience',
 'siutation_analysis',
 'service_search_term',
 'services_plain_text',
 'thinking_through_service_rankings',
 'service_id_list',
 'service_ids',
 'offers',
 'recurring_formula_parsed',
 'initial_formula_parsed',
 'recurring_logic',
 'initial_logic',
 'recurring_price',
 'initial_price',
 'pitch']

def observe_steps(steps, start, num_steps):
    step_list = [
        f'Inputs: Customer Issues: {steps["customer_issue"]}\nCustomer Preferences: {steps["customer_preferences"]}\nProperty Type: {steps["property_type"]}\nSquare Footage: {steps["square_footage"]}\nAcres: {steps["acres"]}\nTarget: {steps["target"]}\nSeverity: {steps["severity"]}',
        f'situational_experience: {steps["situational_experience"]}',
        f'siutation_analysis: {steps["siutation_analysis"]}',
        f'service_search_term: {steps["service_search_term"]}',
        f'services_plain_text: {steps["services_plain_text"]}',
        f'thinking_through_service_rankings: {steps["thinking_through_service_rankings"]}',
        f'service_id_list: {steps["service_id_list"]}',
        f'service_ids: {steps["service_ids"]}',
        f'offers: {steps["offers"]}',
        f'recurring_formula_parsed: {steps["recurring_formula_parsed"]}',
        f'recurring_logic: {steps["recurring_logic"]}',
        f'recurring_price: {steps["recurring_price"]}',
        f'initial_formula_parsed: {steps["initial_formula_parsed"]}',
        f'initial_logic: {steps["initial_logic"]}',
        f'initial_price: {steps["initial_price"]}',
        f'pitch: {steps["pitch"]}'
    ]

    end = start + num_steps
    list = step_list[start:end]
    for i in list:
        print(i)
        print('-----------------')

